In [ ]:
#Installing the necessary library
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found 

In [ ]:
#importing libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bertopic import BERTopic
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')

2025-05-18 21:52:44.324880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747605164.645434      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747605164.739781      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/n

True

Before modeling the topic, we lemmatize the sentences in order to remove, semantically identical words.

In [ ]:
sentence_df = pd.read_csv("/kaggle/input/dataframe/sentence_df.csv")

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

sentence_df["lemmatized_sentence"] = sentence_df["cleaned_sentence"].apply(lemmatize_text)
sentence_df.head()

,source,url,title,publish_date,sentence,lang,cleaned_sentence,sentiment_score,sentiment_label,lemmatized_sentence
0,web,https://allafrica.com/stories/202412050450.html,Rwanda: Commuters All Smiles As Kigali Launche...,2024-12-05 12:12:05+00:00,Commuters were all smiles on Wednesday Decembe...,en,commuters were all smiles on wednesday decembe...,0.5267,positive,commuter were all smile on wednesday december ...
1,web,https://allafrica.com/stories/202412050450.html,Rwanda: Commuters All Smiles As Kigali Launche...,2024-12-05 12:12:05+00:00,"The new system, which began with the Kabuga-Do...",en,the new system which began with the kabugadown...,0.0000,neutral,the new system which began with the kabugadown...
2,web,https://allafrica.com/stories/202412050450.html,Rwanda: Commuters All Smiles As Kigali Launche...,2024-12-05 12:12:05+00:00,ALSO READ: Public transport: Kigali to pilot d...,en,also read public transport kigali to pilot dis...,0.4767,positive,also read public transport kigali to pilot dis...
3,web,https://allafrica.com/stories/202412050450.html,Rwanda: Commuters All Smiles As Kigali Launche...,2024-12-05 12:12:05+00:00,Here's how it works.,en,heres how it works,0.0000,neutral,here how it work
4,web,https://allafrica.com/stories/202412050450.html,Rwanda: Commuters All Smiles As Kigali Launche...,2024-12-05 12:12:05+00:00,"When boarding the bus, you must first select y...",en,when boarding the bus you must first select yo...,0.4019,positive,when boarding the bus you must first select yo...


In [ ]:
#We add the lemmatized sentences into a list
documents = sentence_df['lemmatized_sentence'].dropna().tolist()

In [ ]:
#We create a model
topic_model = BERTopic(language="english", calculate_probabilities=True,verbose=True,embedding_model = 'paraphrase-MiniLM-L3-v2')

#model fitting
topics, probs = topic_model.fit_transform(documents)

2025-05-18 21:56:39,114 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2025-05-18 21:56:50,556 - BERTopic - Embedding - Completed ✓
2025-05-18 21:56:50,556 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-18 21:57:00,952 - BERTopic - Dimensionality - Completed ✓
2025-05-18 21:57:00,954 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-18 21:57:00,986 - BERTopic - Cluster - Completed ✓
2025-05-18 21:57:01,000 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-18 21:57:01,047 - BERTopic - Representation - Completed ✓


In [ ]:
#we extract topics
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

Number of topics: 6


,Topic,Count,Name,Representation,Representative_Docs
0,-1,70,-1_to_the_and_it,"[to, the, and, it, they, is, of, on, system, w...",[if a passenger come from kabuga via sonatube ...
1,0,193,0_the_in_to_transport,"[the, in, to, transport, and, kigali, of, for,...",[all these factor must be considered the cost ...
2,1,20,1_tap_card_the_to,"[tap, card, the, to, when, their, and, out, me...",[to ensure accurate billing passenger will be ...
3,2,16,2_email_you_we_the,"[email, you, we, the, your, processing, sent, ...",[almost finished we need to confirm your email...
4,3,16,3_rwf_per_from_km,"[rwf, per, from, km, kilometre, of, to, and, c...",[generally taxi from the airport should cost r...


The above table has 3 main columns, providing information about all the topics in descending order of topics size/Count.

- 'Topic' is the topic number and the outliers are labeled as -1. Those are topics that should be ignored because they do not bring any added value.
- 'Count' is the number of words in the topic.
- 'Name' is the name given to the topic.
For each topic, we can retrieve the top words and their corresponding c-TF(Term Frequency: How often a word appears in a document)-IDF(Inverse Document Frequency: How unique a word is across all documents) score. Overall we want to evaluate the score by using c-TF-IDF to evaluate the most representative words for each class/topic rather than for each document.

The higher the score, the most relevant the word is in representing the topic.

In [ ]:

a_topic = freq.iloc[1]["Topic"] # Select the 1st topic
topic_model.get_topic(a_topic) # Show the words and their c-TF-IDF scores

[('the', 0.08155805831823226),
 ('in', 0.05775171025974202),
 ('to', 0.0576582505072392),
 ('transport', 0.052086092974183235),
 ('and', 0.0491716527326922),
 ('kigali', 0.04325960020427289),
 ('of', 0.041529959624685066),
 ('for', 0.039907842405786154),
 ('fare', 0.03955962773788519),
 ('public', 0.038681696970361557)]

We have words like "transport", "fare", "public", "kigali" which are coherent with the distance-based fare pricing topic that we are studying.

In [ ]:
b_topic = freq.iloc[2]["Topic"] # Select the 2nd topic
topic_model.get_topic(b_topic) # Show the words and their c-TF-IDF scores

[('tap', 0.14913144700995784),
 ('card', 0.1451934021791053),
 ('the', 0.11525367254699401),
 ('to', 0.07430718000683256),
 ('when', 0.07016183355559319),
 ('their', 0.06982195507945195),
 ('and', 0.06186502203507332),
 ('out', 0.06000846641427328),
 ('meter', 0.05932135261425083),
 ('boarding', 0.05932135261425083)]

The second topic has the second highest count of words if we exclude the outliers. And here we have words like "tap", "card", "boarding" which also aligns very well with our topic of study.

In [ ]:
#Most relevant topic themes
topic_model.visualize_barchart(top_n_topics=5)

The most relevant words of each topic can be visualized in a form of barchart out of the c-TF-IDF score. On the graph above, we represent the corresponding visualization for the 5 topics. The longer the bar, the most relevant is the topic to our analysis. The keywords with the highest c-TF-IDF score accross our 5 topics are "the", "tap", "email", "rwf", "user" which are particularly relevant to our analysis.

In [ ]:
#Intertopic distance map
topic_model.visualize_topics()

We have an interactive dashboard that shows the corresponding words and scores for each topic and the distance between topics in a 2D representation.

In [ ]:
topic_model.visualize_documents(documents)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

The topics are represented as centroids in different clusters (characterized by different colors), just to show that BERT extracted the most relevant information.

Now we can search topics

In [ ]:
# Select most 3 similar topics
similar_topics, similarity = topic_model.find_topics("fare", top_n = 3)

In [ ]:
# topics index from most similar to least similar.
similar_topics

[-1, 0, 1]

In [ ]:
most_similar = similar_topics[0]
print("Most Similar Topic Info: \n{}".format(topic_model.get_topic(most_similar)))
print("Similarity Score: {}".format(similarity[0]))

Most Similar Topic Info: 
[('to', 0.08412295571086012), ('the', 0.07439556080312539), ('and', 0.05361701253519879), ('it', 0.0467694552004219), ('they', 0.040312328019012536), ('is', 0.04000483225817386), ('of', 0.03875698113956234), ('on', 0.03561797656388653), ('system', 0.03359262191420848), ('will', 0.032257193430226816)]
Similarity Score: 0.479356586933136


We searched for the top 3 topics that are semantically similar to "fare" using the find_topics function. We can see that the most similar topic is showing term 'system', which is related to fare.

In [ ]:
similar_topics_2, similarity_2 = topic_model.find_topics("price", top_n = 3)
most_similar_2 = similar_topics_2[0]
print("Most Similar Topic Info: \n{}".format(topic_model.get_topic(most_similar_2)))
print("Similarity Score: {}".format(similarity_2[0]))

Most Similar Topic Info: 
[('rwf', 0.37044403755291366), ('per', 0.14894448775284982), ('from', 0.09908034879695361), ('km', 0.09543129415480055), ('kilometre', 0.08433496504692947), ('of', 0.07695695769486995), ('to', 0.06805207836880352), ('and', 0.06742685561622047), ('cost', 0.05951157330304514), ('lot', 0.05556276970346228)]
Similarity Score: 0.4239168167114258


Here, we searched for the top 3 topics that are semantically similar to "price". We can see that the most similar topic is showing terms like 'rwf', "kilometer", "cost", which are indeed related to "price".

In conclusion, the goal of our topic modeling was to uncover the main themes in public reactions to Rwanda's shift to distance-based public transport fares. Diverse public opinions regarding the fare reform were found through topic modeling.  Although many praised the new system's fairness and modernism, serious issues with price and fare structure confusion still exist.  To increase public pleasure and trust, these insights can direct future policy changes and communication tactics.